# 斐波纳契内存使用

我们讨论 *what* 与*how* 编程和解决性能 *memoization* 的缺陷。

我们使用经典的斐波那契数列作为最初的例子

     0,1,1,2,3,5,8,13,21,34,55 ...
    
我们现在完成一个使用 memoization 的例子，以尽量减少重复的Web请求

## Fib

斐波那契数列通常递归地定义如下

              /            0              if i is 0
    fib(i) =  |            1              if i is 1
              \ fib(i - 1) + fib(i - 2)   otherwise

因为Python支持递归，我们可以把这个数学定义干净地转换成代码。

In [1]:
def fib_what(i):
    if i == 0:
        return 0
    elif i == 1:
        return 1
    else:
        return fib_what(i -1) + fib_what(i - 2)

map(fib_what, range(11))

用来计算第i个斐波那契数的常见算法是保持一对数字的运算，将它们相加得到下一个数字

A common *algorithm* for *how* to compute the $i^{th}$ Fibonacci number efficiently is to keep a running pair of numbers, summing them together to obtain the next

In [2]:
def fib_how(i):
    a, b = 0, 1
    for _ in range(i):
        a, b = b, a + b
    return a


map(fib_how, range(11))


这两个实现在两个方面有所不同。

**Style: What v. How**：

* 第一个函数类似于一个数学定义，定义*什么*将被计算，而没有明确指定* how *。 这个算法的实现取决于Python运行时如何处理递归调用。

* 第二个函数提供了* how *来计算答案的清晰配方; 但是从函数定义中不能立即明白什么是被计算的。

*what* 的解决方案往往比 *how* 的实现概念上更清晰。

**性能**

不幸的是，*what* 的解是非常的缓慢。 这是常见的 *what* 代码。

These two implementations differ in two ways.  

**Style: What v. How**:  

*   The first function is similar to a mathematical definition, defining *what* is to be computed without clearly specifying about *how*.  The implementation of this algorithm depends on how the Python runtime handles recursive calls.

*   The second function provides a clear recipe for *how* to compute the answer; it is not immediately clear from the function definition *what* is being computed however.

*What* solutions tend to be more conceptually clear than *how* solutions.  

**Performance**

Unfortunately the *what* solution is painfully slow.  This is common with *what* code.

In [3]:
%timeit fib_what(35)

6.23 s ± 238 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%timeit fib_how(35)

3.45 µs ± 170 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


注意单位。 什么解决方案比这个输入慢两百万倍。 它只会变得更糟。

*what* 的解决方案比 *how* 的解决方案需要更多的时间。 这是因为它一次又一次地重复调用相同值的“fib”。

示例:

*   `fib(5)` calls `fib(4)` and `fib(3)`
*   `fib(4)` calls `fib(3)` and `fib(2)`
*   `fib(3)` calls `fib(2)` and `fib(1)`
*   The second `fib(3)` also calls `fib(2)` and `fib(1)`
*   All three `fib(2)`s call `fib(1)` and `fib(0)`
*   We now have five redundant calls to `fib(1)`. 

对于像`5`这样的简单输入，性能上来说有些滑稽。 对于大的输入值，它迅速变成悲剧。


## Caching

通过缓存中间值可以保存 *what* 解决方案。

In [5]:
cache = dict()

def fib(i):
    if i in cache:                # Have we seen this input before?
        return cache[i]           # if so then return the cached result
    
    if i == 0:
        return 0
    if i == 1:
        return 1
    else:
        result = fib(i -1) + fib(i - 2)
        cache[i] = result         # Remember that fib(i) == result
        return result

%timeit cache.clear(); fib(35)

25 µs ± 935 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


通过将结果缓存计算的函数中，我们减少了计算时间，但增加了内存使用量。这通常是一个有用空间换时间技术解决方案。


## Memoization

从历史上看，相对比 *how* 函数式语言更有利于实现 *what*。

考虑我们熟悉的 `map`。 `map`函数定义*在完全隐藏的时候*应该做什么*如何完成。 我们可以用各种有趣的方式自由地实现“地图”。 功能抽象将计算机控制从程序员手中带走，并将其交付给语言和基础设施设计人员。

因为函数式语言有利于他们经常遇到的问题，比如我们在斐波纳契范例中看到的重复计算。 而不是明确地缓存我们所做的每一个功能，我们把缓存变成一个更高阶的函数。

这个函数被称为“memoize”。



In [6]:
from toolz import memoize

def fib(i):
    if i == 0:
        return 0
    elif i == 1:
        return 1
    else:
        return fib(i -1) + fib(i - 2)

cache = dict()
fib = memoize(fib, cache)

%timeit cache.clear(); fib(35)

82.7 µs ± 9.08 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


在上面的例子中，我们明确地创建了一个缓存，以便在每个时间点之前清除它。 通常我们只需要像这样使用一个参数来调用memoize

    fib = memoize(fib)
    
并自动生成缓存字典。

或者，你可能更喜欢使用Python装饰器语法

```
@memoize
def fib(i):
    if i == 0:
        return 0
    if i == 1:
        return 1
    else:
        return fib(i -1) + fib(i - 2)
```

这两个都是相同的。

Memoization提供了一个纯粹的 *how* 优化，以便我们可以继续无耻地写数学清晰 *what* 代码。



## 使用Memoization避免显式状态

斐波纳契的例子是一个典型的memoization用例。 可悲的是，我们很少要求在现实生活中计算斐波纳契数。

在这里，我们玩一个更现实的例子。 我们想按照他们内容中不同单词的数量对一组网页进行排序。



In [7]:
# We'll use the Wikipedia articles for the fifty United States

with open('data/states.txt') as f:
    urls = ['http://en.wikipedia.org/wiki/' + state for state in f.read().strip().split()]

### Exercise

我们的任务是通过网站文本中出现的不同词汇的数量对这些网址进行排序。 我们将通过`urllib`中`sorted`函数的关键字`cmp`和`urlopen`来做到这一点。

我们将通过一个明确缓存性能结果的实现。 您将通过适当地添加“memoize”来简化这个实现。


In [8]:
# Naive Solution

# We can get the raw HTML from a URL with `urlopen`
# Depending on your internet connection this might take a little while
import urllib2
opener = urllib2.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]  # Trick Wikipedia into thinking that we're not a robot
urlopen = opener.open


def count_distinct_words(url):
    """ Count distinct words of a URL
    
    >>> count_distinct_words('http://www.example.com')
    86
    """
    text = urlopen(url).read()
    return len(set(text.split()))

            
def cmp_url(a, b):
    """ A comparator function for URLs
    
    >>> cmp_url('http://en.wikipedia.org/', 'http://www.example.org')
    1
    """
    return count_distinct_words(a) - count_distinct_words(b)


# sorted(urls, cmp=cmp_url)  # We could do this but it would be slow

ModuleNotFoundError: No module named 'urllib2'

In [47]:
# Efficient Solution

# To increase performance and reduce the extent to which we hammer the Wikipedia servers 
# we'll precompute all of the counts ahead of time in one pass 
# and store them in a global dictionary

word_counts = dict()
for url in urls:
    word_counts[url] = count_distinct_words(url)


def cmp_url_cached(a, b):
    """ A Cached version of ``cmp_url`` """
    return cmp(word_counts[a], word_counts[b])
          

# sorted(urls cmp=cmp_url_cached)

KeyboardInterrupt: 

### Task

你的任务很简单。 在我们原有的解决方案上使用memoization 功能，以达到与高效解决方案相同的性能。

## End

性能建立往往需要建立状态。 Memoization捕获了更高性能代码的常用策略，特别是在避免重复昂贵的操作时。 “memoize”高阶函数将这个功能包装成一个简单的函数调用或装饰器。